# Importing The Libraries

In [1]:
# Core Libraries.
import pandas as pd
import numpy as np
import datetime as dt

#Visualization
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#Precison Handling settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 160)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading and Reading the Dataset 

In [2]:
df = pd.read_csv(r"C:\Users\HP\OneDrive - National Economics University\Study NEU\DBA\Project 2\dataset.csv")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.000,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12.000,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12.000,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.000,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.000,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


# Overview of Dataset

In [3]:
df.shape

(779495, 8)

In [4]:
df.dtypes.value_counts()

object     4
float64    3
int64      1
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      779495 non-null  int64  
 1   StockCode    779495 non-null  object 
 2   Description  779495 non-null  object 
 3   Quantity     779495 non-null  float64
 4   InvoiceDate  779495 non-null  object 
 5   Price        779495 non-null  float64
 6   Customer ID  779495 non-null  float64
 7   Country      779495 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 47.6+ MB


# Feature Engineering

In [6]:
#Creating new feature Revenue

df["Revenue"] = df["Quantity"] * df["Price"]

In [7]:
#Changing data type to datetime

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [8]:
print("Max date:", df["InvoiceDate"].max())
print("Min date:", df["InvoiceDate"].min())

Max date: 2011-12-09 12:50:00
Min date: 2009-12-01 07:45:00


## RFM Featuring

In [9]:
#Set Latest date 2011-12-10 as last invoice date was 2011-12-09.
Latest_Date = dt.datetime(2011,12,10)

In [10]:
#Creating RFM features with subset of customerID

RFM = df.groupby('Customer ID').agg({'InvoiceDate': lambda x: (Latest_Date - x.max()).days, 
                                                 'Invoice': lambda x: x.nunique(), 
                                                 "Revenue": lambda x: x.sum()})

In [11]:
RFM['InvoiceDate'] = RFM['InvoiceDate'].astype(int)

In [12]:
#Renaming column names to Recency, Frequency and Monetary

RFM.rename(columns={'InvoiceDate': 'Recency', 
                         'Invoice': 'Frequency', 
                         'Revenue': 'Monetary'}, inplace=True)

In [13]:
RFM.reset_index().head()

,Customer ID,Recency,Frequency,Monetary
0,12346.000,325,12,745.700
1,12347.000,2,8,4921.530
2,12348.000,75,5,1991.860
3,12349.000,18,4,3786.500
4,12350.000,310,1,331.340


### New Feature: InterPurchase Time

The Fourth varibale of RFM, InterPurchase Time, is a measure of average time gap between total shopping trips by a customer. The Interpurchase Time is calcluted as fallows :

$$T = \frac{L}{F-1} = \frac{T_n - T_1}{F-1}$$

- $T$: Interpurchase Time
- $L$: Shopping Cycle
- $F$: Frequency
- $T_1$: First purchase
- $T_n$: Last purchase

Note: We consider only those customers who made purchase more than once.

In [14]:
RFM = RFM[(RFM["Frequency"]>1)]

In [15]:
RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.000,325,12,745.700
12347.000,2,8,4921.530
12348.000,75,5,1991.860
12349.000,18,4,3786.500
12352.000,36,10,2098.940


In [16]:
Shopping_Cycle = df.groupby('Customer ID').agg({'InvoiceDate': lambda x: ((x.max() - x.min()).days)})

In [17]:
RFM["Shopping_Cycle"] = Shopping_Cycle

In [18]:
RFM.head()

,Recency,Frequency,Monetary,Shopping_Cycle
Customer ID,,,,
12346.000,325,12,745.700,400
12347.000,2,8,4921.530,402
12348.000,75,5,1991.860,362
12349.000,18,4,3786.500,570
12352.000,36,10,2098.940,356


In [19]:
RFM["Interpurchase_Time"] = RFM["Shopping_Cycle"] // RFM["Frequency"]

In [20]:
RFMT = RFM[["Recency", "Frequency", "Monetary", "Interpurchase_Time"]]

In [21]:
RFMT.head()

,Recency,Frequency,Monetary,Interpurchase_Time
Customer ID,,,,
12346.000,325,12,745.700,33
12347.000,2,8,4921.530,50
12348.000,75,5,1991.860,72
12349.000,18,4,3786.500,142
12352.000,36,10,2098.940,35


In [22]:
RFMT.shape

(4255, 4)

In [23]:
RFMT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4255 entries, 12346.0 to 18287.0
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Recency             4255 non-null   int32  
 1   Frequency           4255 non-null   int64  
 2   Monetary            4255 non-null   float64
 3   Interpurchase_Time  4255 non-null   int64  
dtypes: float64(1), int32(1), int64(2)
memory usage: 149.6 KB
